# Brownian motion

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
# Bokeh libraries
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, WheelZoomTool, Legend, Span
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.palettes import Spectral11, Blues, viridis
from bokeh.io import output_notebook
output_notebook(hide_banner =  True)

In the early 1800's, the botanist Robert Brown studied the random movement of pollen particles suspended in water. Since practically anyone with a microscope could observe this phenomena, he is not credited with its discovery. However, he reserched the topic extensively, and he ruled out the hypothesis that the effect was life-related. For his work on the topic, this random movement and yiggling of particles is now called Brownian motion.

To introduce Brownian motion, we show below a video of nanoparticles (small particles) in water, as we can see, they jiggle randomly. This random jiggling is what we call Brownian motion, and we thus call these particles Brownian particles.

In [10]:
%%HTML
<video width="400" controls>
  <source src="../_static/lecture_specific/brownian_motion/brownian_motion_nanoparticles_in_water-rutger_saly_youtube.mp4" type="video/mp4">
</video>

<a href="https://www.youtube.com/watch?v=cDcprgWiQEY">
<br>(video source)
</a>

We can observe similar phenomena in the movements on dust particles floating on air. The phenomenon continued to be studied by several scientists. By 1905, the kinetic theory seemed the most plausible explanation for Brownian motion. This theory states that fluids, like gas and water, are composed of a large number of identical tiny particles in constant and rapid motion that are constantly colliding with each other. Then, the Brownian motion of a pollen grain or a dust particle in a fluid could be explained by the movement resulting from the constant collision against the fluid particles. Although this was conceptually correct, theoretical attempts to describe experimental observations kept failing.

In 1905, Einstein's publishes in one of his Annus Mirabilis papers and solves the problem. He developed the statistical mechanics and the molecular-kinetic theory of thermodynamics. He did this mostly unaware of the existence of the phenomenon and previous studies. The theory was correct and described consistently the experimental observations of Brownian motion. It was a breakthrough! If you are interested in more historical details, we recommend to read the first chapters of {cite}`nelson2020dynamical`.

## Simulating Brownian motion
Although the movement of Brownian particles is random, we can ask ourselves if there is anything predictable about their movement? If there is some predictable behavior, maybe there is also predictable behavior in other random systems, such as the economy, social networks and molecular biology. 

To start, we would like to describe the movement of Brownian particles with a mathematical model. As the formal theory of Brownian motion is quite involved, a more accesible approach is to use computational simulations. One reason Einstein's approach succesfully describe Brownian motion is because it focuses on the positions of particles while ignoring their velocities. Mimicking his spirit, consider a Brownian particle with its position $x(t)$ given by a function of time $t$. We assume the particle only moves in one dimension, so it can only move left and right. Given the position at time $t=0$, we would like to calculate its position at a later time, say $t=1$ later. As Brownian motion is random, we expect to add a random number $R$,

$$
x(1) = x(0) + R.
$$

We can apply this formula again and obtain the position at time $t=2$, i.e $x(2)$. In general, for any given $t$, we can simply write

$$
x(t+1) = x(t) + R
$$

However, we were still not sure what random number $R$ we should use. If you observe Brownian particles, they are likely to stay in the same area, small movements seem much more likely than large ones. Furthermore, it seems every direction is equally likely, so a natural candidate is a normal distribution with mean zero. If you don't know what is a normal distribution, check the [random numbers chapter](./basic_concepts/random_numbers). How likely it is to go farther away, can be modulated by the constant $\sigma$. We thus write the equation for the time evolution of the position of the Brownian particle as

$$
x(t+1) = x(t) +  \sigma \mathcal{N}(0,1),
$$ 

where $\mathcal{N}(0,1)$ denote a random number sampled from a normal distribution with mean $0$ and standard deviation $1$. With this equation we can already simulate our first Brownian trajectory and plot it as a function of time.

In [ ]:
# One dimensional simulation of one Brownian particle
sigma = 1
max_steps = 250
xlist=[0]
for i in range(max_steps):
    xcurrent = xlist[i] 
    xnew = xcurrent + sigma * np.random.normal(0,1)
    xlist.append(xnew)

# Plot position as function of time
plt.plot(xlist, color='darkslateblue', alpha=0.8, lw=2, label="Brownian trajectory")
plt.xlim([0,max_steps])
plt.xlabel("time")
plt.ylabel("position")
plt.legend();

In the next interactive plot, we plot up to 200 trajectories. Move the slider to reduce or increase the number of trajectories being plotted. Do you see any pattern?

In [ ]:
# Bokeh interactive plot for many 1D Brownian trajectories
sigma = 1
max_steps = 250
num_trajectories = 200
trajectories = []
for n in range(num_trajectories):
    # One dimensional simulation of one Brownian particle
    xlist=[0]
    for i in range(max_steps):
        xcurrent = xlist[i] 
        xnew = xcurrent +  sigma * np.random.normal(0,1)
        xlist.append(xnew)
    trajectories.append(xlist)

time = np.array([range(max_steps)]*num_trajectories)

mypalette = viridis(num_trajectories)
alpha_array = [0.5]*num_trajectories

source = ColumnDataSource(data=dict(t = time, trajs = trajectories, palette = mypalette, opacity = alpha_array))

plot = figure(x_range=(0, max_steps), plot_width=450, plot_height=300, tools="pan,wheel_zoom,reset") 
plot.title.text = '1D Brownian trajectories'
plot.title.text_font_size = '12pt'
plot.toolbar.logo = None

plot.xaxis.axis_label = 'time'
plot.yaxis.axis_label = 'position'

plot.multi_line(xs="t", ys='trajs', line_color='palette', line_alpha='opacity', line_width=2, source=source)

num_trajs_slider = Slider(start=1, end=num_trajectories, 
                          value=num_trajectories, step=1, title="Number of trajectories")

callback = CustomJS(args=dict(source=source, num_trajs=num_trajs_slider),
                    code="""
    const data = source.data;
    const opacity = data['opacity']
    const max_trajs = num_trajs.value;
    for (var i = 0; i < opacity.length; i++) {
        if (i < max_trajs) {
            opacity[i] = 0.5
        } else {
            opacity[i] = 0.0
        }
    }
    source.change.emit();
""")

num_trajs_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(num_trajs_slider),
)

show(layout)

Indeed it seems particles have a tendency to spread outwards while still having their average position close to their stating position, i.e close to zero. We will continue exploring the one-dimensional model in the next section, but to finlize this section, lets look to something closer to reality. We can easily extend this model to two or more dimensions by sampling a random number the same way for each additional axis. This way we can simulate a Brownian trajectory in two dimensions and plot it

In [ ]:
# Two dimensional simulation of one Brownian particle
mu = 0
sigma = 1
dimension = 2
max_steps = 401
positions = np.zeros([max_steps, 2]) # initial position (0,0)
sigma = 1
for t in range(max_steps - 1):
    positions[t+1] = positions[t] + sigma * np.random.normal(mu,1,dimension)

where the function `np.random.normal(mu,sigma,dimension)` now accepts a third argument corresponding to the dimension. This function outputs two independently sampled normal numbers, one for each dimension. We can use this data to produce an interactive plot. You can adjust the time slider to see the movement of the Brownian particle through time.

In [ ]:
# Bokeh interactive plot code for 2D trajectory
minval = np.min(positions)
maxval = np.max(positions)

xpositions = positions[:,0]
ypositions = positions[:,1]

source = ColumnDataSource(data=dict(x=xpositions, y=ypositions))
original = ColumnDataSource(data=dict(x=xpositions, y=ypositions))
source_dot = ColumnDataSource(data=dict(x=[xpositions[-1]], y=[ypositions[-1]]))


plot = figure(x_range=(minval, maxval), y_range=(minval, maxval), plot_width=400, 
              plot_height=400, tools="pan,wheel_zoom,reset") #toolbar_location=None)
plot.title.text = '2D Brownian trajectory'
plot.title.text_font_size = '12pt'
plot.toolbar.logo = None

trajectory = plot.line('x', 'y', source=source, line_width=2, line_alpha=0.6, color='#1f1f7a')
particle = plot.circle('x', 'y', source=source_dot, size = 10, alpha=0.6, color="red")

legend = Legend(items=[('Brownian particle', [particle]), 
                       ('Brownian trajectory', [trajectory])], 
                orientation="horizontal")
plot.add_layout(legend, 'below')
legend.click_policy="hide"


time_slider = Slider(start=0, end=max_steps-1, value=max_steps-1, step=1, title="Time")

callback = CustomJS(args=dict(source=source, source_dot=source_dot, original=original, time=time_slider),
                    code="""
    const data = source.data;
    const data_dot = source_dot.data;
    const original_data = original.data
    const t = time.value;
    const x = data['x']
    const y = data['y']
    const xtrue = original_data['x']
    const ytrue = original_data['y']
    const xdot = data_dot['x']
    const ydot = data_dot['y']
    for (var i = 0; i < x.length; i++) {
        if (i < t) {
            x[i] = xtrue[i]
            y[i] = ytrue[i]
        } else {
            x[i] = xtrue[t]
            y[i] = ytrue[t]
        }
    }
    xdot[0] = xtrue[t] 
    ydot[0] = ytrue[t] 
    source.change.emit();
    source_dot.change.emit();
""")

time_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(time_slider),
)

show(layout)

Note every time the code is simulated the trajectory will be different since it is resampled every time. Now try to zoom out and move the time slider. To zoom out activate the zoom function in the interactive controls next to the plot. Zoom ot more and try again. Does the movement of the particle look similar to the movement of the particles in the initial video?

## Emergent collective behavior

In this section, we will show how a collective behavior emerges from the individual movement of the Brownian particles. One can mathematically show the collective behavior of the Brownian particles in one dimension follows
 
$$
\frac{\partial \rho}{\partial t} = D\frac{\partial^2 \rho(x)}{\partial x^2},
$$

where $\rho(x,t)$ is the distribution of particles in $x$ at a given time $t$. If you don't understand this equation, don't worry; it requires more advanced mathematics beyond the scope of this book. It is a partial differential equation, and it can model several different physical process, such as the dynamics of heat and the diffusion of fluids; it is thus called the heat equation or the difussion equation. The value of $D$ is called the diffusion coefficient, and it is related to the value of the standard deviation of our simulations above by 

$$
D=\sigma^2/2.
$$ 

The form of the equation is not important, we just care that it exists and that it has a solution, which is a function of position and time. Assuming all the particles are concentrated at $x=0$ at time 0, the emergent behavior is given by the solution of this equation, which is

$$
\rho(x,t) = \frac{1}{ \sqrt{4\pi D t}} \exp \Big( -\frac{x^2}{4Dt}\Big).
$$

Given a value of time $t$, say $t=100$, we can easily plot this function as a function of $x$. If what we discussed is correct, this should be the distribution that our Brownian particles should obey at time $t=100$.

In [ ]:
x = np.linspace(-50, 50, 100)
time = 100
sigma = 1
D = sigma*sigma/2
distribution = 1/np.sqrt(4 * np.pi * D * time) * np.exp(-x*x/(4 * D * time))
plt.plot(x, distribution, lw =2, color='orange')
plt.xlabel('x')
plt.ylabel(r'$\rho(x,t=$' +str(time) + ')')
plt.title('Solution of diffusion equation')
plt.xlim([-50,50]);

To verify the emergent collective behavior of the particles is indeed of this form, we can simply simulate a bunch of Brownian particles and check how they are distributed after a given time. We thus follow the next steps:

1. Simulate a large number of identical Brownian particles in one dimension.
2. Create a histogram of the distribution of Brownian particles at each point in time.
3. Plot the histogram of the distribution for a given time.

We write two python functions to facilitate doing these steps. Try to read the code and the comments to understand what each function does. The first function calculates a Brownian trajectory given the initial position of the particle, the corresponding value of $\sigma$ and the total number of time iterations (the length of the trajectory). The second function, given a time and the trajectories of many Brownian particles, extracts the positions of all the particles at a given time.

In [ ]:
def calculate_brownian(xinitial, sigma, maxsteps):
    '''
    Function to calculate the trajectory of one Brownian particle in one dimension 
    
    :param xinitial: initial position of trajectory
    :param sigma: standard deviation of normal dsitribution to sample random number
    :param maxsteps: total number of time steps taken
    :return: brownian trajectory in the form of an array with each entry the 
    position at the corresponding time step
    '''
    xlist=[xinitial]
    for i in range(maxsteps):
        xcurrent = xlist[i] 
        xnew = xcurrent + sigma * np.random.normal(0,1)
        xlist.append(xnew)
    return np.array(xlist)

def extract_positions(timestep, brownian_trajs):
    '''
    Function to extract positions of all the Brownian particles at a given time step
    
    :param timestep: timestep at which to extract positions (equal to time in this notebook)
    :param brownian_trajs: array of brownian trajectories, where each entry is the
    complete trajectory of a brownian particle
    :return: array containing the positions of all Brownian particles at the given timestep
    '''
    positions = []
    numparticles = len(brownian_trajs)
    for i in range(numparticles):
        positions.append(brownian_trajs[i][timestep])
    return positions

We can use these functions to calculate and plot the distribution of many Brownian particles at any given time.

In [ ]:
x0 = 0 # initial position for all particles
sigma = 1
max_steps = 250
num_particles = 2000

# Calculates trajetory of all Brownian particles and saves data in brownian_trajectories
brownian_trajectories = []
for i in range(num_particles):
    brownian_trajectory = calculate_brownian(x0, sigma, max_steps)
    brownian_trajectories.append(brownian_trajectory)

# Plot the histogram of the distribution of Brownian particles
time = 100
positions = extract_positions(time, brownian_trajectories)
plt.hist(positions, 30, density=True, alpha=0.5, color='dodgerblue', edgecolor='black', lw=0.3)
plt.xlabel('x')
plt.ylabel('Distribution histogram (t=' + str(time) + ')')
plt.title('Distribution of Brownian particles')
plt.xlim([-50,50]);

Now we want to compare the two results. We do this all at once and interactively to really show how the histograms calculated from the positions of Brownian particles at any given time obeys the corresponding distribution given by the diffusion equation.

In [ ]:
sigma = 1
D = sigma*sigma/2
xdomain = np.linspace(-50, 50, 100)
hist_list = []
leftedges_list = []
rightedges_list = []
pdf_list = []
for i in range(max_steps):
    time = i+1
    positions_particles = extract_positions(time, brownian_trajectories)
    hist, edges = np.histogram(positions_particles, density=True, bins=30)
    hist_list.append(hist)
    leftedges_list.append(edges[:-1])
    rightedges_list.append(edges[1:])
    pdf = 1/np.sqrt(4 * np.pi * D * time) * np.exp(-xdomain*xdomain/(4 * D * time))
    pdf_list.append(pdf)


sourceHist = ColumnDataSource(data=dict(hist_data= hist_list[max_steps-1], 
                                        leftedges_data = leftedges_list[max_steps-1],
                                        rightedges_data = rightedges_list[max_steps-1]))
sourcePDF = ColumnDataSource(data=dict(x=xdomain, pdf_data=pdf_list[max_steps-1]))
sourceMain = ColumnDataSource(data=dict(hist_data= hist_list, 
                                        leftedges_data = leftedges_list,
                                        rightedges_data = rightedges_list,
                                        pdf_data = pdf_list))



plot = figure(x_range=(-50,50), y_range=(0,0.2), plot_width=400, plot_height=400, tools="pan,wheel_zoom,reset")
plot.title.text = 'Distribution of 1D Brownian trajectories'
plot.title.text_font_size = '12pt'
plot.toolbar.logo = None

histogram = plot.quad(top='hist_data', bottom=0, left='leftedges_data', 
                      right='rightedges_data', source = sourceHist, alpha=0.6, color='#0066ff')
distribution = plot.line('x', 'pdf_data', source = sourcePDF, line_color="orange", line_width=4, alpha=0.7)

legend = Legend(items=[('Distribution of Brownian particles', [histogram]), 
                       ('Diffusion equation solution', [distribution])])
plot.add_layout(legend)
legend.click_policy="hide"

plot.xaxis.axis_label = 'x'
plot.yaxis.axis_label = 'Pr(x)'

time_slider = Slider(start=1, end=max_steps, value=max_steps, step=1, title="Time")

callback = CustomJS(args=dict(sourceHist=sourceHist, sourcePDF=sourcePDF, sourceMain=sourceMain, time=time_slider),
                    code="""
    const dataHist = sourceHist.data;
    const dataPDF = sourcePDF.data;
    const dataMain = sourceMain.data;
    
    const t = time.value - 1;
    const x = dataPDF['x'];
    const pdf = dataPDF['pdf_data'];
    const hist_data = dataHist['hist_data'];
    const leftedges_data = dataHist['leftedges_data'];
    const rightedges_data = dataHist['rightedges_data'];
    
    const pdf_main = dataMain['pdf_data'];
    const hist_data_main = dataMain['hist_data'];
    const leftedges_data_main = dataMain['leftedges_data'];
    const rightedges_data_main = dataMain['rightedges_data'];

    
    for (var i = 0; i < x.length; i++) {
        pdf[i] = pdf_main[t][i]
    }
    for (var i = 0; i < hist_data.length; i++) {
        hist_data[i] = hist_data_main[t][i];
        leftedges_data[i] = leftedges_data_main[t][i];
        rightedges_data[i] = rightedges_data_main[t][i];
    }
        
    sourceHist.change.emit();
    sourcePDF.change.emit();
""")

time_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(time_slider),
)

show(layout)

This shows the emergent behavior from the colective motion of Brownian particles follows a very predictable form given by the diffusion equation. As a final representation of Brownian motion, we present below an interactive plot of many one dimensional Brownian trajectories as a function of time, together with its corresponding distribution. One can vary the total number of trajectories used to calculate the distirbution and the time at which it is calculated. It clearly shows one needs a large enough sample to generate a reasonable distribution.  

In [ ]:
# Bokeh interactive plot for many 1D Brownian trajectories
sigma = 1
max_steps = 300
stepsize = 10
num_trajectories = 250
steptrajs = 10
trajectories = []
x0 = 0
times = np.array([range(max_steps)]*num_trajectories)

for n in range(num_trajectories):
    # One dimensional simulation of one Brownian particle
    trajectory = calculate_brownian(x0, sigma, max_steps)
    trajectories.append(trajectory)
    
# Calculates all possible histograms
hist_all = []
edgesbottom_all = []
edgestop_all = []
# For any possible number of trajectories
for j in range(int(num_trajectories/steptrajs) + 1):
    hist_array = []
    edgesbottom_array = []
    edgestop_array = []
    maxtrajs_plot = j*steptrajs
    # For any possible time
    for i in range(int(max_steps/stepsize)+1):
        time_i = i * stepsize
        if j>0:
            positions = extract_positions(time_i, trajectories[0: maxtrajs_plot])
            hist, edges = np.histogram(positions, bins=30, density=True)
            hist_array.append(hist)
            edgesbottom_array.append(edges[:-1])
            edgestop_array.append(edges[1:])
        else:
            hist_array.append([])
            edgesbottom_array.append([])
            edgestop_array.append([])
    hist_all.append(hist_array)
    edgesbottom_all.append(edgesbottom_array)
    edgestop_all.append(edgestop_array)

mypalette = viridis(num_trajectories)
alpha_array = [0.5]*num_trajectories

source = ColumnDataSource(data=dict(t = times, trajs = trajectories, palette = mypalette, opacity = alpha_array))
sourceHist = ColumnDataSource(data=dict(hist = hist_all[-1][-1], 
                                        edgesbottom = edgesbottom_all[-1][-1], 
                                        edgestop = edgestop_all[-1][-1]))
sourceMaster = ColumnDataSource(data=dict(histarray = hist_all, 
                                          edgesbottomarray = edgesbottom_all,
                                          edgestoparray = edgestop_all))

# Plot trajectories
plot_trajs = figure(x_range=(0, max_steps-0.01), y_range=(-45, 45), plot_width=400, plot_height=300, 
              tools="pan,wheel_zoom,reset", toolbar_location="above") 
plot_trajs.title.text = '1D Brownian trajectories'
plot_trajs.title.text_font_size = '12pt'
plot_trajs.toolbar.logo = None
plot_trajs.xaxis.axis_label = 'time'
plot_trajs.yaxis.axis_label = 'position'
plot_trajs.multi_line(xs="t", ys='trajs', line_color='palette', line_alpha='opacity', line_width=2, source=source)
# Plot time indicator line
span = Span(location=max_steps, dimension='height', line_color='black', line_dash='dashed', line_width=2)
plot_trajs.renderers.extend([span])

# Plot distribution (histogram)
plot_hist = figure(toolbar_location=None, plot_width=250, plot_height=plot_trajs.plot_height, 
                   x_range=(0, 0.1), y_range=plot_trajs.y_range, y_axis_location=None)
plot_hist.xaxis.major_label_orientation = np.pi/4
plot_hist.title.text = 'Distribution of particles'
plot_hist.title.text_font_size = '12pt'
plot_hist.quad(left=0, bottom='edgesbottom', top='edgestop', right='hist', alpha=0.5, 
               color='#0066ff', source=sourceHist)

num_trajs_slider = Slider(start=0, end=num_trajectories, 
                          value=num_trajectories, step=steptrajs, title="Number of trajectories")
time_slider = Slider(start=0, end=max_steps, value=max_steps, step=stepsize, title="Time")

callback = CustomJS(args=dict(span=span, source=source, sourceHist=sourceHist, sourceMaster=sourceMaster, 
                                   time = time_slider, num_trajs=num_trajs_slider, 
                                   stepsize = stepsize, steptrajs=steptrajs),
                    code="""
    const data = source.data;
    const dataHist = sourceHist.data;
    const dataMaster = sourceMaster.data;
    
    const opacity = data['opacity']
    const hist_data = dataHist['hist'];
    const edgesbottom_data = dataHist['edgesbottom'];
    const edgestop_data = dataHist['edgestop'];
    const hist_data_master = dataMaster['histarray']
    const edgesbottom_data_master = dataMaster['edgesbottomarray']
    const edgestop_data_master = dataMaster['edgestoparray']
    
    const t = time.value
    const max_trajs = num_trajs.value
    const tint = t/stepsize
    const maxtjint = max_trajs/steptrajs
    
    for (var i = 0; i < opacity.length; i++) {
        if (i < max_trajs) {
            opacity[i] = 0.5
        } else {
            opacity[i] = 0.0
        }
    }    
    for (var i = 0; i < hist_data.length; i++) {
        hist_data[i] = hist_data_master[maxtjint][tint][i]
        edgesbottom_data[i] = edgesbottom_data_master[maxtjint][tint][i]
        edgestop_data[i] = edgestop_data_master[maxtjint][tint][i]
    }
        
    span.location = t 
    
    span.change.emit()
    source.change.emit();
    sourceHist.change.emit();
""")


num_trajs_slider.js_on_change('value', callback)
time_slider.js_on_change('value', callback)


layout = column(
    row(
    plot_trajs,
    plot_hist),
    column(num_trajs_slider, time_slider),
)

show(layout)

```{Note}
One final note about the equation we are solving. We used this equation to simulte Brownian motion

$$
x(t+1) = x(t) + \sigma \mathcal{N}(0,1)  \\[5mm]
$$

A more general equation would use a continuous variable for time. This means looking at $x(t+\Delta t)$ instead of $x(t+1)$. This would modify the equation to

$$
x(t+\Delta t) = x(t) + \sqrt{\Delta t}\sigma\mathcal{N}(0,1).
$$

This can also be written in terms of the diffusion coefficient by using that $D=\sigma^2/2$,

$$
x(t+\Delta t) = x(t) + \sqrt{2D \Delta t}\mathcal{N}(0,1).
$$

The addition of $\sqrt{\Delta t}$  and the relation $D=\sigma^2/2$ are not obvious, and they are closely related to the essence of stochastic calculus. Actually these last equations corresponds to the discretization of the stochastic differential equation

$$
dX_t = \sigma dW_t,
$$

which is the equation that describes standard Brownian motion. The required mathematics to understand this equation are beyond the scope of this book, but the interested reader can investigate further on its own. Our simulation of Brownian trajectories presented above corresponds to a numerical scheme used to solve this equation. The scheme is called Euler-Maruyama, and it is introduced in great detail on {cite}`higham2001algorithmic`.
```

## Concluding remarks

- We introduced the dynamics of individual Brownian particles through a simulation scheme, and we showed how it can be extended to two (or more) dimensions. We hint at the connection to the field of stochastic differential equations, which is the mathematically correct approach to model Brownian motion. 

- Using computational simulations of Brownian motion, we showed that the collective behavior of a large number of Brownian particles obeys the diffusion equation. The diffusion equation has no random components, and thus it is deterministic. In other words, despite Brownian motion being random, its collective behavior is predictable. The emergence of the diffusion equation from the dynamics of a large number of Brownian particles can actually be proved mathematically.

Before Einstein's work, the scientific community was divided about the existence of atoms. Thanks to Einstein's theory of Brownian motion along with other physical advances at the time convinced the sceptics, who were quite numerous at that time (Ostwald, Mach), of the reality of atoms {cite}`nelson2020dynamical`. Let me rephrase this, around 100 and some years ago, we were not even sure of the existence of atoms. In a 100 years from now, what other scientific facts will be trivial that are still unknown today. 

The studies of Brownian motion not only provided a physical theory to describe Brownian movement; they also gave birth to the field of stochastic differential equations. These equations describe trajectories that are nowhere differentiable, such as idealized Brownian trajectories, and thus require a different type of calculus  called stochastic calculus. Furthermore, each system of stochastic differential equations has a corresponding emergent "diffusion" equation called the Fokker-Planck equation. This yields a fundamental duality of the same underlying process: the stochastic dynamics of individual particles/agents together with the probabilistc dynamics of an ensemble of identical particles/agents. Today, these equations are used in applications ranging from molecular biology to financial mathematics.

We should note this was just a brief and incomplete introduction to Brownian motion. There are many other important personalities, such as Smoluchowski and Langevin, involved in the development of Brownian motion. There are also many results and important theoretical details ommited. If the reader is interested in learning more about the topic, we recommend checking the references below.

## Suggested projects

We present a list of suggested topics, feel free to change the projects and explore different directions. This should not be seen as excercises for a class but as exploratory projects. Try writing your projects in a Jupyter notebook. This way you can combine your explanations, your plots and code in one place.

```{admonition} Project 1: Ornstein–Uhlenbeck process (difficulty: 4/5)
:class: tip, dropdown

**a.** The equation for the time evolution of the position of a Brownian particle is $ x(t+1) = x(t) + \sigma \mathcal{N}(0,1)$ (since we assumed $\Delta t = 1$). Add a term of the form $-\theta x(t)$ with theta a positive constant, then modify the code from the notes and implement a simulation of its trajectory in one dimension starting at $x=0$. Use the parameters $\theta=0.01$ and $\sigma = 1$. Plot the position as a function of time. Can you notice anything different? If you don't, try plotting the solution with a different initial position $x_0$ and compare it to the standard Brownian motion case ($\theta =0$). 

**b.** The solution of the Fokker-Planck equation for the Ornstein–Uhlenbeck process from exercise 3 has the form: 

$$
\rho(x,t) = \sqrt{\frac{\theta}{2\pi D (1-e^{-2\theta t})}} \exp{\Bigg(-\frac{\theta x^2}{2D (1-e^{-2\theta t})} \Bigg)},
$$

assuming all the particles/probability are concentrated at $x=0$ at time $t=0$.

Numerically show how a large collection of simulated trajectories (done in a.) produce an emergent behavior by plotting a histogram of the particles distribution. Remember we used $\sigma =1$ and $\theta =0.01$. Check this behavior matches the one given by the solution to the corresponding Fokker-Planck equation. (Hint: Copy and modify the code used in the notes.) How is the behavior different from the Brownian motion simulations presented in the text?
```

```{admonition} Project 2: Diffusion under a field of force (difficulty: 4/5)
:class: tip, dropdown

The Brownian particle can also be subject to an external force such as gravity or other. Some forces $F$ can be written as the derivative of a function called the potential function, $U(x)$, as follows: $F=-dU(x)/dx$. This force can easily be incorporated into our Brownian simulation equation,

$$
x(t+\Delta t) =  x(t) -\Delta t \frac{dU(x)}{dx} + \sqrt{2D\Delta t}\mathcal{N}(0,1).
$$

Note this time we are using the equation for an arbitrary time interval $\Delta t$. Assume the potential function has the form $U(x)=-x^2 +x^4/16$. (Also note that to make this force term physically consistent, we would need an additional factor.)

**a.** Plot the potential function from $x=-5$ to $x=5$. Check what is the direction of the force (positive or negative) in the different regions of the plot. For this you only need to know if the derivative of $U(x)$ is positive or negative (remember $F=-dU(x)/dx$) The minima of this function represent the sites to where the particle is being pushed to.

**b.** Using the paramters $D=1$, $\Delta t = 0.01$, generate a Brownian trajectory following the equation above with $100000$ timesteps. Plot the Brownian trajectory as a function of time. What do you observe? Do you observe a relation of the Brownian trajectory to the potential function? Discuss your findings.

```

```{admonition} Project 3: The Einstein relation (difficuly 2/5)
:class: tip, dropdown

The Einstein relation relates the diffusion of Brownian particles quantified by the diffusion coefficient $D$ with the dissipation quantified by the friction coefficient $\gamma$ and the temperature $T$ by the relation

$$
D = \frac{k_B T}{\gamma}.
$$

The constant $k_B$ is called the Boltzmann constant. To keep things simple, assume $k_B=1$. This diffusion constant appears in the diffusion equation 

$$
\frac{\partial \rho}{\partial t} = D\frac{\partial^2 \rho(x)}{\partial x^2},
$$

**a.** Plot the solutions of the diffusion equation for $T=1$ and $\gamma=1$ interactively as a function of time. (Hint copy and paste the code in the text above)

**b.** Choose a fixed time. What happens when you replot the solution using of $T>1$ or $T<1$? Analogously, what happens when you replot the solution using $\gamma>1$ and $\gamma>1$? Present some plots that illustrate your findings.

**c.** Try to give a physical explanation why you observe the changes you see when increasing or decreasing the friction or the temperature.
```

## References

```{bibliography} ../_bibliography/references.bib
:style: unsrt
:filter: docname in docnames
```